# OptOut Project: Constructing Independent Variables for Subsamples for 14-year-seq sample

* Post-1979 Sample: mothers with 1 year between first birth month and interview month in 1979 wave
* Married Sample: was married by our definition, excluding cases with valid skip in both spouse earning and spouse hours in the survey year of child birth
* Working Sample: respondents who provided job information in the last survey that is at least a year prior to the birth, provided it is not more than 3 years prior to the birth.

In [1]:
#setup
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
from operator import itemgetter
from datetime import datetime, timedelta

# set some nicer defaults for matplotlib
from matplotlib import rcParams

rcParams['figure.figsize'] = (10, 6)
rcParams['figure.dpi'] = 150
rcParams['lines.linewidth'] = 2
rcParams['axes.grid'] = False
rcParams['axes.facecolor'] = 'white'
rcParams['font.size'] = 14
rcParams['patch.edgecolor'] = 'none'

def remove_border(axes=None, top=False, right=False, left=True, bottom=True):
    """
    Minimize chartjunk by stripping out unnecessary plot borders and axis ticks
    
    The top/right/left/bottom keywords toggle whether the corresponding plot border is drawn
    """
    ax = axes or plt.gca()
    ax.spines['top'].set_visible(top)
    ax.spines['right'].set_visible(right)
    ax.spines['left'].set_visible(left)
    ax.spines['bottom'].set_visible(bottom)
    
    #turn off all ticks
    ax.yaxis.set_ticks_position('none')
    ax.xaxis.set_ticks_position('none')
    
    #now re-enable visibles
    if top:
        ax.xaxis.tick_top()
    if bottom:
        ax.xaxis.tick_bottom()
    if left:
        ax.yaxis.tick_left()
    if right:
        ax.yaxis.tick_right()

In [2]:
# helper functions

def diff_month(d1, d2):
    """
    Return the number of months between the two dates
    """
    
    return (d1.year - d2.year) * 12 + d1.month - d2.month
    
    
def list_duplicates_of(seq, item):
    """
    Given a seq and item, return the indexes of duplicate items in the list (as a list).
    (found online)
    """
    start_at = -1
    locs = []
    while True:
        try:
            loc = seq.index(item,start_at+1)
        except ValueError:
            break
        else:
            locs.append(loc)
            start_at = loc
    return locs

# Week-to-week crosswalk

Create two functions to convert between week number and calendar year and month

In [3]:
#this is the file to switch between week number in NLSY79 and calendar year, month, and day
xl_file = pd.ExcelFile('../data/continuous_week_crosswalk_r26.xlsx') 
weekdf = xl_file.parse('weekdates1')
weekdf.head()

,Week Start: Month,Week Start: Day,Week Start: Year,Calendar Year Week Number,Continuous Week Number
0,1,1,1978,1,1
1,1,8,1978,2,2
2,1,15,1978,3,3
3,1,22,1978,4,4
4,1,29,1978,5,5


In [4]:
def get_week_num(year, month, which, df=weekdf):
    """
    Given a calendar year and month, return the corresponding week number in NLSY79.
    
    If which=='first', return the first week in that month.
    If which=='last', return the last week in that month. 
    """
    
    the_month = df[(df['Week Start:\nMonth']==month)&(df['Week Start: \nYear']==year)]
    
    if which == 'first':
        return int(the_month.iloc[0]['Continuous \nWeek Number'])
    elif which == 'last':
        return int(the_month.iloc[len(the_month)-1]['Continuous \nWeek Number'])

In [5]:
def get_year_month(weeknum, df=weekdf):
    """
    Given a week number, return the corresponding calendar month and year (yyyy, m).
    """
    
    the_week = df[df['Continuous \nWeek Number']==weeknum]
    return int(the_week.iloc[0]['Week Start: \nYear']), int(the_week.iloc[0]['Week Start:\nMonth'])

In [6]:
# read entire sample
iv_df = pd.read_csv('../data/iv_df_seq14.csv') 
print iv_df.shape
iv_df.head()

(3688, 24)


,caseid_1979,childbirth_year,childbirth_month,y_dob,m_dob,race,age,educ,marstat,mom_worked,...,num_ft,num_pt1,num_pt2,num_work_miss_hr,num_unemp,num_oolf,num_nonwork,num_ml,num_miss,num_nonemp_spells
0,2,1993,3,1959,1,3,34,2,1.0,0.0,...,1,0,43,0,25,98,0,1,0,1
1,3,1981,6,1961,8,3,19,1,1.0,1.0,...,37,6,0,0,17,108,0,0,0,7
2,4,1980,8,1962,8,3,17,1,0.0,1.0,...,86,4,0,0,12,66,0,0,0,5
3,8,1976,3,1958,7,3,17,1,1.0,1.0,...,91,3,8,0,12,17,1,15,21,3
4,16,1990,2,1958,10,3,31,3,1.0,0.0,...,41,67,50,0,0,0,0,10,0,0


## Identify Post-1979 Sample - women with birth at least 1 year after 1979 interview 

In [7]:
intdate_df = pd.read_csv('../data/int_date.csv')
intdate_df.head()

,caseid_1979,date_m_1979,date_d_1979,date_m_1980,date_d_1980,date_m_1981,date_d_1981,date_m_1982,date_d_1982,date_m_1983,...,date_y_2008,date_d_2010,date_m_2010,date_y_2010,date_d_2012,date_m_2012,date_y_2012,date_d_2014,date_m_2014,date_y_2014
0,1,3,3,-5,-5,6,25,-5,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1,2,2,28,3,3,2,23,2,6,2,...,2008,10,3,2010,3,12,2012,26,12,2014
2,3,2,8,5,20,-5,-5,3,11,3,...,2008,-5,-5,-5,19,3,2013,8,5,2015
3,4,2,8,-5,-5,3,24,3,3,1,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
4,5,4,19,4,3,3,12,3,1,4,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5


In [8]:
def is_subsample(row):
    """
    Return True if respondent is in Post-1979 Sample --- 
    women with birth at least 1 year after 1979 interview
    """
    
    int_row = intdate_df[intdate_df.caseid_1979==row['caseid_1979']].iloc[0]
    
    d = diff_month(datetime(int(row['childbirth_year']), int(row['childbirth_month']), 1), 
                   datetime(1979, int(int_row['date_m_1979']), 1))
    
    return int(d >= 12)
    
    
iv_df['post1979_sample'] = iv_df.apply(is_subsample, axis=1)
iv_df.post1979_sample.value_counts()

1    2706
0     982
Name: post1979_sample, dtype: int64

In [9]:
iv_df[iv_df.post1979_sample==1].childbirth_year.value_counts()

1981    264
1982    264
1980    239
1983    233
1985    218
1984    214
1986    180
1989    151
1988    150
1987    137
1990    124
1991    100
1992     90
1993     87
1994     58
1995     49
1996     43
1997     39
1998     28
1999     22
2000     14
2001      2
Name: childbirth_year, dtype: int64

In [10]:
sub_iv_df = iv_df[iv_df.post1979_sample==1]

## Whether to work at age 35

Variables Used:
    
exp_1_1979: WHAT WOULD R LIKE TO BE DOING AT AGE 35?

* 1 PRESENT JOB
* 2 SOME OCCUPATION
* 3 MARRIED, FAMILY
* 4 OTHER (SPECIFY)

exp_2_1979: DOES R WANT TO WORK OUTSIDE HOME AT AGE 35?
* 1 YES
* 0 NO

In [11]:
# read attitude variables
att_df = pd.read_csv('../data/attitude.csv')
att_df.head()

,caseid_1979,fer_3_1979,womens_roles_000001_1979,womens_roles_000002_1979,womens_roles_000003_1979,womens_roles_000004_1979,womens_roles_000005_1979,womens_roles_000006_1979,womens_roles_000007_1979,womens_roles_000008_1979,...,exp_1_1981,exp_2_1981,womens_roles_000001_1982,womens_roles_000002_1982,womens_roles_000003_1982,womens_roles_000004_1982,womens_roles_000005_1982,womens_roles_000006_1982,womens_roles_000007_1982,womens_roles_000008_1982
0,1,2,2,2,1,1,4,2,1,4,...,1,-4,-5,-5,-5,-5,-5,-5,-5,-5
1,2,3,4,3,2,2,3,4,3,4,...,2,1,3,3,2,2,4,4,3,4
2,3,4,2,4,3,3,3,3,3,3,...,-5,-5,1,1,1,1,4,1,4,1
3,4,2,1,2,3,4,1,2,4,2,...,1,-4,1,1,4,2,4,2,4,2
4,5,3,2,2,2,1,3,3,3,2,...,1,-4,2,2,2,1,3,3,3,2


In [12]:
# crosstab
merged = sub_iv_df.merge(att_df, on='caseid_1979')
pd.crosstab(merged.exp_1_1979, merged.exp_2_1979)

exp_2_1979,-4,-3,0,1
exp_1_1979,,,,
-3,2,0,0,0
-2,95,0,0,0
1,34,0,0,0
2,1875,0,0,0
3,0,19,215,388
4,78,0,0,0


In [13]:
def working35(i):
    """
    Given respondent id i, return whether respondent would like to work at age 35.
    
    Value scheme:
    0 - answered MARRIED, FAMILY & NO to the question DOES R WANT TO WORK OUTSIDE HOME AT AGE 35?
    2 - answered Don't Know. 
    1 - the rest, except missings
    """
    
    row = att_df[att_df.caseid_1979==i].iloc[0]
    
    if row['exp_1_1979']==-2: # answered Don't Know
        return 2
    
    elif row['exp_1_1979'] in [1, 2, 4]:
        return 1
    
    elif row['exp_1_1979']==3 and row['exp_2_1979']==1:
        return 1
    
    elif row['exp_1_1979']==3 and row['exp_2_1979']==0:
        return 0
    
    return None # capture missing


sub_iv_df['work35'] = iv_df.caseid_1979.map(working35)
sub_iv_df.head()

/Users/xzhuo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,caseid_1979,childbirth_year,childbirth_month,y_dob,m_dob,race,age,educ,marstat,mom_worked,...,num_pt2,num_work_miss_hr,num_unemp,num_oolf,num_nonwork,num_ml,num_miss,num_nonemp_spells,post1979_sample,work35
0,2,1993,3,1959,1,3,34,2,1.0,0.0,...,43,0,25,98,0,1,0,1,1,1.0
1,3,1981,6,1961,8,3,19,1,1.0,1.0,...,0,0,17,108,0,0,0,7,1,0.0
2,4,1980,8,1962,8,3,17,1,0.0,1.0,...,0,0,12,66,0,0,0,5,1,0.0
4,16,1990,2,1958,10,3,31,3,1.0,0.0,...,50,0,0,0,0,10,0,0,1,1.0
5,19,1987,11,1957,12,3,29,2,1.0,0.0,...,0,0,0,168,0,0,0,1,1,1.0


In [14]:
sub_iv_df.work35.value_counts(dropna=False)

 1.0    2375
 0.0     215
 2.0      95
NaN       21
Name: work35, dtype: int64

## Women's role

In [15]:
#what are the possible answers for each role question?

for i in [1, 2, 4, 6, 8]:
    print 'question', i
    print att_df['womens_roles_00000%s_1979'%i].unique()

question 1
[ 2  4  1  3 -2 -3]
question 2
[ 2  3  4  1 -2 -3 -1]
question 4
[ 1  2  3  4 -2 -3]
question 6
[ 2  4  3  1 -3 -2]
question 8
[ 4  3  2  1 -2 -3 -1]


In [16]:
count_missing_only_one = []

def get_wm_roles(i):
    """
    Given a respondent id i, return the summary index of women's roles. 
    
    If one variable has missing value, expand its value.
    If more than one missing, return None.
    """
    
    row = att_df[att_df.caseid_1979==i].iloc[0]
    
    num_missing = sum([row['womens_roles_00000%s_1979'%i] < 0 for i in [1, 2, 4, 6, 8]])
    
    if num_missing == 0: # no missing
        return np.mean([int(row['womens_roles_00000%s_1979'%i]) for i in [1, 2, 4, 6, 8]])
    
    elif num_missing == 1:
        count_missing_only_one.append(i)
        vals = [int(row['womens_roles_00000%s_1979'%i]) for i in [1, 2, 4, 6, 8] 
                if int(row['womens_roles_00000%s_1979'%i]) > 0]
        return np.mean(vals)
    
    else:
        return None
        

sub_iv_df['womensroles'] = sub_iv_df.caseid_1979.map(get_wm_roles)
sub_iv_df.head()

/Users/xzhuo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,caseid_1979,childbirth_year,childbirth_month,y_dob,m_dob,race,age,educ,marstat,mom_worked,...,num_work_miss_hr,num_unemp,num_oolf,num_nonwork,num_ml,num_miss,num_nonemp_spells,post1979_sample,work35,womensroles
0,2,1993,3,1959,1,3,34,2,1.0,0.0,...,0,25,98,0,1,0,1,1,1.0,3.4
1,3,1981,6,1961,8,3,19,1,1.0,1.0,...,0,17,108,0,0,0,7,1,0.0,3.0
2,4,1980,8,1962,8,3,17,1,0.0,1.0,...,0,12,66,0,0,0,5,1,0.0,2.2
4,16,1990,2,1958,10,3,31,3,1.0,0.0,...,0,0,0,0,10,0,0,1,1.0,2.2
5,19,1987,11,1957,12,3,29,2,1.0,0.0,...,0,0,168,0,0,0,1,1,1.0,2.2


In [17]:
len(count_missing_only_one)

71

## Ideal number of children

This variable was reported in 1979 and 1982. After comparing values from the two survey years, we decide to use the value reported in 1979. 

In [18]:
ideal_df = pd.read_csv('../data/ideal_num_child.csv')
ideal_df.head()

,caseid_1979,ideal_num_children_1979,ideal_num_children_1982
0,1,3,-5
1,2,3,3
2,3,3,2
3,4,3,2
4,5,3,2


In [19]:
# use variable reported in 1979
sub_iv_df = sub_iv_df.merge(ideal_df[['caseid_1979', 'ideal_num_children_1979']], on='caseid_1979')
sub_iv_df.rename(columns={'ideal_num_children_1979': 'ideal'}, inplace=True) #rename columns

def recode_ideal(x):
    """
    Recode the variable ideal to integers or None. Topcode at 4. 
    """
    
    if x < 0:
        return None
    elif x > 4: # topcoded at 4
        return 4
    else:
        return int(x)
    
sub_iv_df.ideal = sub_iv_df.ideal.map(recode_ideal)
sub_iv_df.head()

,caseid_1979,childbirth_year,childbirth_month,y_dob,m_dob,race,age,educ,marstat,mom_worked,...,num_unemp,num_oolf,num_nonwork,num_ml,num_miss,num_nonemp_spells,post1979_sample,work35,womensroles,ideal
0,2,1993,3,1959,1,3,34,2,1.0,0.0,...,25,98,0,1,0,1,1,1.0,3.4,3.0
1,3,1981,6,1961,8,3,19,1,1.0,1.0,...,17,108,0,0,0,7,1,0.0,3.0,3.0
2,4,1980,8,1962,8,3,17,1,0.0,1.0,...,12,66,0,0,0,5,1,0.0,2.2,3.0
3,16,1990,2,1958,10,3,31,3,1.0,0.0,...,0,0,0,10,0,0,1,1.0,2.2,2.0
4,19,1987,11,1957,12,3,29,2,1.0,0.0,...,0,168,0,0,0,1,1,1.0,2.2,4.0


In [20]:
sum(sub_iv_df.ideal.isnull())

8

## Religion 

R’s report of religious affiliation and frequency of religious attendance in 1979

* Affiliation we want to code into 4 categories: Protestant, Catholic, other, none;
* Frequency of attendance we want to code into 3 categories: < monthly, monthly but not weekly; weekly+

In [21]:
rel_df = pd.read_csv('../data/religion.csv')
rel_df.head()

,caseid_1979,rel_col_1979,rel_1979,rel_freq_1979
0,1,7,7,5
1,2,7,7,5
2,3,7,7,2
3,4,7,7,1
4,5,7,7,1


### Religious affiliation - use rel_col_1979

* 1 - none; 
* 2 - Catholic; 
* 3 - Protestant - this includes not just the Protestant code, but Baptist, Episcopalian, Lutheran, Methodist, and Presbyterian; 
* 4 - Something else - this will include Jewish, too.

NLSY codes:
* 0 NONE, NO RELIGION
* 1 PROTESTANT
* 2 BAPTIST
* 3 EPISCOPALIAN
* 4 LUTHERAN
* 5 METHODIST
* 6 PRESBYTERIAN
* 7 ROMAN CATHOLIC
* 8 JEWISH
* 9 OTHER

In [22]:
merged = sub_iv_df.merge(rel_df, on='caseid_1979')
merged.rel_col_1979.value_counts()

 7    956
 2    704
 9    306
 0    189
 5    185
 4    126
 1    108
 6     68
 3     34
 8     21
-3      7
-2      2
Name: rel_col_1979, dtype: int64

In [23]:
def get_rel(i):
    rel = rel_df[rel_df.caseid_1979==i].iloc[0]['rel_col_1979']
    
    if rel == 0:
        return 1
    elif rel == 7:
        return 2
    elif rel in [1, 2, 3, 4, 5, 6]:
        return 3
    elif rel in [8, 9]:
        return 4
    elif rel < 0:
        return None
    
sub_iv_df['religion'] = sub_iv_df.caseid_1979.map(get_rel)
sub_iv_df.head()

,caseid_1979,childbirth_year,childbirth_month,y_dob,m_dob,race,age,educ,marstat,mom_worked,...,num_oolf,num_nonwork,num_ml,num_miss,num_nonemp_spells,post1979_sample,work35,womensroles,ideal,religion
0,2,1993,3,1959,1,3,34,2,1.0,0.0,...,98,0,1,0,1,1,1.0,3.4,3.0,2.0
1,3,1981,6,1961,8,3,19,1,1.0,1.0,...,108,0,0,0,7,1,0.0,3.0,3.0,2.0
2,4,1980,8,1962,8,3,17,1,0.0,1.0,...,66,0,0,0,5,1,0.0,2.2,3.0,2.0
3,16,1990,2,1958,10,3,31,3,1.0,0.0,...,0,0,10,0,0,1,1.0,2.2,2.0,2.0
4,19,1987,11,1957,12,3,29,2,1.0,0.0,...,168,0,0,0,1,1,1.0,2.2,4.0,2.0


In [24]:
sub_iv_df.religion.value_counts(dropna=False)

 3.0    1225
 2.0     956
 4.0     327
 1.0     189
NaN        9
Name: religion, dtype: int64

## Frequency of attendence
* 1 - < monthly
* 2 - monthly but not weekly
* 3 - weekly+


NLSY codes:
* 1 NOT AT ALL
* 2 INFREQUENTLY
* 3 ONCE PER MONTH
* 4 2-3 TIMES PER MONTH
* 5 ONCE PER WEEK
* 6 > ONCE PER WEEK

In [25]:
merged.rel_freq_1979.value_counts()

 5    784
 2    652
 4    369
 6    340
 1    332
 3    225
-3      2
-1      2
Name: rel_freq_1979, dtype: int64

In [26]:
def get_rel_freq(i):
    freq = rel_df[rel_df.caseid_1979==i].iloc[0]['rel_freq_1979']
    
    if freq in [1, 2]:
        return 1
    elif freq in [3, 4]:
        return 2
    elif freq in [5, 6]:
        return 3
    elif freq < 0:
        return None
    
    
sub_iv_df['religion_freq'] = sub_iv_df.caseid_1979.map(get_rel_freq)
sub_iv_df.head()

,caseid_1979,childbirth_year,childbirth_month,y_dob,m_dob,race,age,educ,marstat,mom_worked,...,num_nonwork,num_ml,num_miss,num_nonemp_spells,post1979_sample,work35,womensroles,ideal,religion,religion_freq
0,2,1993,3,1959,1,3,34,2,1.0,0.0,...,0,1,0,1,1,1.0,3.4,3.0,2.0,3.0
1,3,1981,6,1961,8,3,19,1,1.0,1.0,...,0,0,0,7,1,0.0,3.0,3.0,2.0,1.0
2,4,1980,8,1962,8,3,17,1,0.0,1.0,...,0,0,0,5,1,0.0,2.2,3.0,2.0,1.0
3,16,1990,2,1958,10,3,31,3,1.0,0.0,...,0,10,0,0,1,1.0,2.2,2.0,2.0,1.0
4,19,1987,11,1957,12,3,29,2,1.0,0.0,...,0,0,0,1,1,1.0,2.2,4.0,2.0,1.0


In [27]:
sub_iv_df.religion_freq.value_counts(dropna=False)

 3.0    1124
 1.0     984
 2.0     594
NaN        4
Name: religion_freq, dtype: int64

## Hours worked between 2y and 1y prebirth and bewteen 1y and 1m prebirth 

from NLSY79 variables - hours worked in past calendar year

* first birth 1995 or 1997: has 2y-1y hours, but missing 1y hours
* first birth 1996: has no 2y-1y hours, has 1y hours

In [28]:
hours_df = pd.read_csv('../data/hours_worked_past_yr.csv')
hours_df.head()

,caseid_1979,pct_missing_1979,hours_1979,pct_missing_1980,hours_1980,pct_missing_1981,hours_1981,hours_1982,pct_missing_1982,hours_1983,...,hours_2006,pct_missing_2006,hours_2008,pct_missing_2008,hours_2010,pct_missing_2010,hours_2012,pct_missing_2012,hours_2014,pct_missing_2014
0,1,0,1516,-5,-5,0,1976,-5,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1,2,0,1190,0,1820,0,1820,1820,0,1820,...,936,0,884,0,884,0,1820,0,1820,0
2,3,1,-3,0,1440,-5,-5,760,0,0,...,2580,9,60,98,-5,-5,1716,0,2010,0
3,4,0,0,-5,-5,0,0,400,0,50,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
4,5,0,1072,0,832,0,936,628,0,520,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5


In [29]:
# explore missingness

count = defaultdict(int)
count_pct = defaultdict(list)

for i, row in sub_iv_df.iterrows():    
    hours_row = hours_df[hours_df.caseid_1979==row['caseid_1979']].iloc[0]
    birth_y = int(row['childbirth_year'])
    
    if birth_y <= 1994:
        hours = hours_row['hours_%s'%(birth_y-1)]
        pct = hours_row['pct_missing_%s'%(birth_y-1)]
    elif birth_y == 1995 or birth_y == 1997:
        hours = hours_row['hours_%s'%(birth_y-1)]
        pct = hours_row['pct_missing_%s'%(birth_y-1)]

    
    if hours > 0:
        hours = 'valid'
    elif hours == 0:
        pass
    elif hours == -4:
        hours = 'valid skip'
    else:
        hours = 'missing'
        
    count[hours] += 1
    count_pct[hours].append(pct)

In [30]:
for k, v in count.items():
    print k, v

0 310
valid 2161
valid skip 114
missing 121


In [31]:
print np.mean([x>0 for x in count_pct['valid']]), np.mean([x>0 for x in count_pct[0]])

0.0282276723739 0.0161290322581


In [32]:
def get_hours1(row):
    """hours between 2 year and 1 year prebirth"""
    
    hours_row = hours_df[hours_df.caseid_1979==row['caseid_1979']].iloc[0]
    birth_y = int(row['childbirth_year'])
    
    if birth_y <= 1994:
        hours = hours_row['hours_%s'%(birth_y-1)]
        pct = hours_row['pct_missing_%s'%(birth_y-1)]
    elif birth_y in [1995, 1997, 1999, 2001]:
        hours = hours_row['hours_%s'%(birth_y-1)]
        pct = hours_row['pct_missing_%s'%(birth_y-1)]
    elif birth_y in [1996, 1998, 2000]:
        return None

    
    if hours >= 0:
        if pct > 0:
            return None
        return hours
    elif hours == -4:
        return 0
    return None
        
    
sub_iv_df['hours_2y_to_1y'] = sub_iv_df.apply(get_hours1, axis=1)
sub_iv_df.head()

,caseid_1979,childbirth_year,childbirth_month,y_dob,m_dob,race,age,educ,marstat,mom_worked,...,num_ml,num_miss,num_nonemp_spells,post1979_sample,work35,womensroles,ideal,religion,religion_freq,hours_2y_to_1y
0,2,1993,3,1959,1,3,34,2,1.0,0.0,...,1,0,1,1,1.0,3.4,3.0,2.0,3.0,1820.0
1,3,1981,6,1961,8,3,19,1,1.0,1.0,...,0,0,7,1,0.0,3.0,3.0,2.0,1.0,1440.0
2,4,1980,8,1962,8,3,17,1,0.0,1.0,...,0,0,5,1,0.0,2.2,3.0,2.0,1.0,0.0
3,16,1990,2,1958,10,3,31,3,1.0,0.0,...,10,0,0,1,1.0,2.2,2.0,2.0,1.0,2641.0
4,19,1987,11,1957,12,3,29,2,1.0,0.0,...,0,0,1,1,1.0,2.2,4.0,2.0,1.0,138.0


In [33]:
sum(pd.isnull(sub_iv_df.hours_2y_to_1y))

275

In [34]:
def get_hours2(row):
    """hours between 2 year and 1 year prebirth"""
    
    hours_row = hours_df[hours_df.caseid_1979==row['caseid_1979']].iloc[0]
    birth_y = int(row['childbirth_year'])
    
    if birth_y <= 1994 or birth_y in [1996, 1998, 2000]:
        hours = hours_row['hours_%s'%(birth_y)]
        pct = hours_row['pct_missing_%s'%(birth_y)]
        
    elif birth_y in [1995, 1997, 1999, 2001]:
        return None

    
    if hours >= 0:
        if pct > 0:
            return None
        return hours
    elif hours == -4:
        return 0
    return None
        
    
sub_iv_df['hours_1y_to_birth'] = sub_iv_df.apply(get_hours2, axis=1)
sub_iv_df.head()

,caseid_1979,childbirth_year,childbirth_month,y_dob,m_dob,race,age,educ,marstat,mom_worked,...,num_miss,num_nonemp_spells,post1979_sample,work35,womensroles,ideal,religion,religion_freq,hours_2y_to_1y,hours_1y_to_birth
0,2,1993,3,1959,1,3,34,2,1.0,0.0,...,0,1,1,1.0,3.4,3.0,2.0,3.0,1820.0,1820.0
1,3,1981,6,1961,8,3,19,1,1.0,1.0,...,0,7,1,0.0,3.0,3.0,2.0,1.0,1440.0,NaN
2,4,1980,8,1962,8,3,17,1,0.0,1.0,...,0,5,1,0.0,2.2,3.0,2.0,1.0,0.0,NaN
3,16,1990,2,1958,10,3,31,3,1.0,0.0,...,0,0,1,1.0,2.2,2.0,2.0,1.0,2641.0,2005.0
4,19,1987,11,1957,12,3,29,2,1.0,0.0,...,0,1,1,1.0,2.2,4.0,2.0,1.0,138.0,1490.0


In [35]:
sub_iv_df.to_csv('../data/iv_df_seq14_post1979.csv', index=False)

# Get married subsample

(1) she was married 1 month prior to the birth; 
(2) exclude cases in valid skip in both spouse earning and spouse hours in the survey year of child birth (which is 1994 for mothers with birth in 1995)

In [36]:
marstat_df = pd.read_csv('../data/marstat.csv')
marstat_df.head()

,caseid_1979,marstat_1979,marstat_1980,marstat_1981,marstat_1982,marstat_1983,marstat_1984,marstat_1985,marstat_1986,marstat_1987,...,yren2m_xrnd,mobg3m_xrnd,yrbg3m_xrnd,moen3m_xrnd,yren3m_xrnd,marstat_2006,marstat_2008,marstat_2010,marstat_2012,marstat_2014
0,1,0,-5,0,-5,-5,-5,-5,-5,-5,...,-999,-999,-999,-999,-999,-5,-5,-5,-5,-5
1,2,0,0,0,0,0,0,0,0,0,...,-998,-998,-998,-998,-998,1,1,1,1,1
2,3,0,0,-5,1,1,1,1,1,1,...,-998,-998,-998,-998,-998,1,1,-5,1,1
3,4,0,-5,1,1,2,2,2,3,2,...,-996,-996,-996,-996,-996,-5,-5,-5,-5,-5
4,5,0,0,0,0,0,0,-5,1,-5,...,-998,-998,-998,-998,-998,-5,-5,-5,-5,-5


In [37]:
sub_iv_df.childbirth_year.value_counts()

1981    264
1982    264
1980    239
1983    233
1985    218
1984    214
1986    180
1989    151
1988    150
1987    137
1990    124
1991    100
1992     90
1993     87
1994     58
1995     49
1996     43
1997     39
1998     28
1999     22
2000     14
2001      2
Name: childbirth_year, dtype: int64

In [38]:
spouse_df = pd.read_csv('../data/spouse_income.csv')
spouse_df.head()

,version_r26_2014,caseid_1979,spouse_income_1979,sample_id_1979,sample_race_78scrn,sample_sex_1979,spouse_income_1980,spouse_income_1981,spouse_income_1982,spouse_income_1983,...,q13_18_sr000002_trunc_2010,q13_18_uab_a_2010,q13_18_uab_b_2010,q13_18_uab_c_2010,q13_18_trunc_2012,q13_18_sr000001_trunc_2012,q13_18_sr000002_trunc_2012,q13_18_uab_a_2012,q13_18_uab_b_2012,q13_18_uab_c_2012
0,-5,1,-4,5,3,2,-5,-4,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1,458,2,-4,5,3,2,-4,-4,-4,-4,...,-4,-4,-4,-4,0,-4,-4,-4,-4,-4
2,458,3,-4,5,3,2,-4,-5,10000,9600,...,-5,-5,-5,-5,55000,-4,-4,-4,-4,-4
3,-5,4,-4,5,3,2,-5,14000,10000,-4,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
4,-5,5,-4,1,3,1,-4,-4,-4,-4,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5


In [39]:
# merge spouse variables
spouse_hours_df = pd.read_csv('../data/spouse_hours.csv')
spouse_df = spouse_df.merge(spouse_hours_df)
spouse_df.head()

,version_r26_2014,caseid_1979,spouse_income_1979,sample_id_1979,sample_race_78scrn,sample_sex_1979,spouse_income_1980,spouse_income_1981,spouse_income_1982,spouse_income_1983,...,spouse_hr_per_wk_1996,spouse_hr_per_wk_1998,spouse_hr_per_wk_2000,spouse_hr_per_wk_2002,spouse_hr_per_wk_2004,spouse_hr_per_wk_2006,spouse_hr_per_wk_2008,spouse_hr_per_wk_2010,spouse_hr_per_wk_2012,spouse_hr_per_wk_2014
0,-5,1,-4,5,3,2,-5,-4,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1,458,2,-4,5,3,2,-4,-4,-4,-4,...,35,37,40,40,40,-4,-4,-4,-4,-4
2,458,3,-4,5,3,2,-4,-5,10000,9600,...,50,50,40,40,-5,40,40,-5,40,40
3,-5,4,-4,5,3,2,-5,14000,10000,-4,...,40,40,-5,-5,-5,-5,-5,-5,-5,-5
4,-5,5,-4,1,3,1,-4,-4,-4,-4,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5


In [40]:
def is_married_subsample(row):
    spouse_row = spouse_df[spouse_df.caseid_1979==row['caseid_1979']].iloc[0]
    birth_y = int(row['childbirth_year'])
    
    if birth_y <= 1994 or birth_y in [1996, 1998, 2000]:
        income = spouse_row['spouse_income_%s'%birth_y]
        hours = spouse_row['spouse_hr_per_wk_%s'%birth_y]
    elif birth_y in [1995, 1997, 1999, 2001]:
        income = spouse_row['spouse_income_%s'%(birth_y-1)]
        hours = spouse_row['spouse_hr_per_wk_%s'%(birth_y-1)]
    else:
        print 'unexpected', birth_y
    
    return int(row['marstat'] == 1 and not (income == -4 and hours == -4)) 
        

sub_iv_df['married_sample'] = sub_iv_df.apply(is_married_subsample, axis=1)
sub_iv_df.married_sample.value_counts()

1    1767
0     939
Name: married_sample, dtype: int64

In [41]:
married_iv_df = sub_iv_df[sub_iv_df.married_sample==1]
print sub_iv_df.shape, married_iv_df.shape

(2706, 33) (1767, 33)


In [42]:
# conversation rates from January in year to January in another year
# source: http://data.bls.gov/cgi-bin/cpicalc.pl

convert_dict = {
    1978: 3.79,
    1979: 3.47,
    1980: 3.05,
    1981: 2.72,
    1982: 2.51,
    1983: 2.42,
    1984: 2.32,
    1985: 2.25,
    1986: 2.16,
    1987: 2.13,
    1988: 2.05,
    1989: 1.96,
    1990: 1.86,
    1991: 1.76,
    1992: 1.72,
    1993: 1.66,
    1994: 1.62,
    1995: 1.58,
    1996: 1.53,
    1997: 1.49,
    1998: 1.47,
    1999: 1.44,
    2002: 1.34}


def convert_to_current_dollar(x, yr):
    """
    Given an amount of money x and year yr, 
    return the money in 2016 dollar. 
    """
 
    if np.isnan(x) or x == None:
        return None
    elif x == 0:
        return 0
    return x*convert_dict[int(yr)]


In [43]:
# if income == 0, hours are valid skip, assume hours = 0 
# if hours is valid, income is valid skip, assume income is missing

def get_spouse_vars(row):
    
    spouse_row = spouse_df[spouse_df.caseid_1979==row['caseid_1979']].iloc[0]
    birth_y = int(row['childbirth_year'])
    
    adj_income = None
    
    if birth_y <= 1994 or birth_y in [1996, 1998, 2000]:
        income = spouse_row['spouse_income_%s'%birth_y]
        hours = spouse_row['spouse_hr_per_wk_%s'%birth_y]
    elif birth_y == 1995 or birth_y in [1995, 1997, 1999, 2001]:
        income = spouse_row['spouse_income_%s'%(birth_y-1)]
        hours = spouse_row['spouse_hr_per_wk_%s'%(birth_y-1)]
            
    if income == 0 and hours == -4: # assume hours 0 
        return (0, 0, 0)

    elif income == -4 and hours == 0: # assume income missing
        return (0, 0, 0)

    else:
        if income < 0:
            income = None
        if hours < 0:
            hours = None
            
            
        if income >= 0:
            if birth_y <= 1994 or birth_y in [1996, 1998, 2000]:
                adj_income = convert_to_current_dollar(income, birth_y-1)
            elif birth_y in [1995, 1997, 1999, 2001]:
                adj_income = convert_to_current_dollar(income, birth_y-2)
            
        return (income, adj_income, hours)

    
res = married_iv_df.apply(get_spouse_vars, axis=1)
married_iv_df['spouse_earning'] = map(itemgetter(0), res)
married_iv_df['spouse_earning_adj'] = map(itemgetter(1), res)
married_iv_df['spouse_hours'] = map(itemgetter(2), res)
married_iv_df.head()

/Users/xzhuo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/xzhuo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/xzhuo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable

,caseid_1979,childbirth_year,childbirth_month,y_dob,m_dob,race,age,educ,marstat,mom_worked,...,womensroles,ideal,religion,religion_freq,hours_2y_to_1y,hours_1y_to_birth,married_sample,spouse_earning,spouse_earning_adj,spouse_hours
0,2,1993,3,1959,1,3,34,2,1.0,0.0,...,3.4,3.0,2.0,3.0,1820.0,1820.0,1,40000.0,68800.0,35.0
1,3,1981,6,1961,8,3,19,1,1.0,1.0,...,3.0,3.0,2.0,1.0,1440.0,NaN,1,NaN,NaN,NaN
3,16,1990,2,1958,10,3,31,3,1.0,0.0,...,2.2,2.0,2.0,1.0,2641.0,2005.0,1,33000.0,64680.0,60.0
5,20,1990,8,1959,11,3,30,4,1.0,0.0,...,2.4,3.0,2.0,2.0,2080.0,2080.0,1,34000.0,66640.0,40.0
6,25,1990,10,1959,11,3,30,3,1.0,0.0,...,1.8,3.0,1.0,1.0,2080.0,2080.0,1,40000.0,78400.0,55.0


In [44]:
#edit hours into categories:
#0-spouse < FT (less than 35 hours per week), 1-full time (35-50 hours per week), 2-overwork (>50 hours per week)

def group_sphours(h):
    
    if h==None:
        return None
    
    elif np.isnan(h):
        return None

    elif h<35: # less than full-time
        return 0
    
    elif h>=35 and h<=50:
        return 1
    
    elif h>50:
        return 2
    
    else:
        print 'unknown hours'
        return None
        

married_iv_df['spouse_hours_gp'] = married_iv_df.spouse_hours.map(group_sphours)
married_iv_df.head()

/Users/xzhuo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,caseid_1979,childbirth_year,childbirth_month,y_dob,m_dob,race,age,educ,marstat,mom_worked,...,ideal,religion,religion_freq,hours_2y_to_1y,hours_1y_to_birth,married_sample,spouse_earning,spouse_earning_adj,spouse_hours,spouse_hours_gp
0,2,1993,3,1959,1,3,34,2,1.0,0.0,...,3.0,2.0,3.0,1820.0,1820.0,1,40000.0,68800.0,35.0,1.0
1,3,1981,6,1961,8,3,19,1,1.0,1.0,...,3.0,2.0,1.0,1440.0,NaN,1,NaN,NaN,NaN,NaN
3,16,1990,2,1958,10,3,31,3,1.0,0.0,...,2.0,2.0,1.0,2641.0,2005.0,1,33000.0,64680.0,60.0,2.0
5,20,1990,8,1959,11,3,30,4,1.0,0.0,...,3.0,2.0,2.0,2080.0,2080.0,1,34000.0,66640.0,40.0,1.0
6,25,1990,10,1959,11,3,30,3,1.0,0.0,...,3.0,1.0,1.0,2080.0,2080.0,1,40000.0,78400.0,55.0,2.0


In [45]:
married_iv_df.to_csv('../data/iv_df_seq14_married.csv', index=False)

## Working sub-sample

In [46]:
# read datasets
current_df = pd.read_csv('../data/current_at_job.csv')
current_df.head()

,caseid_1979,current_at_job_01_1979,current_at_job_02_1979,current_at_job_03_1979,current_at_job_04_1979,current_at_job_05_1979,current_at_job_01_1980,current_at_job_02_1980,current_at_job_03_1980,current_at_job_04_1980,...,current_at_job_01_2012,current_at_job_02_2012,current_at_job_03_2012,current_at_job_04_2012,current_at_job_05_2012,current_at_job_01_2014,current_at_job_02_2014,current_at_job_03_2014,current_at_job_04_2014,current_at_job_05_2014
0,1,1,0,-4,-4,-4,-5,-5,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1,2,1,-4,-4,-4,-4,1,-4,-4,-4,...,1,-4,-4,-4,-4,1,-4,-4,-4,-4
2,3,1,0,0,-4,-4,1,0,0,-4,...,1,0,-4,-4,-4,1,0,-4,-4,-4
3,4,-4,-4,-4,-4,-4,-5,-5,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
4,5,1,0,-4,-4,-4,1,0,-4,-4,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5


In [47]:
cps_id_df = pd.read_csv('../data/cps_id.csv')
cps_id_df.head()

,caseid_1979,job01_1980,job02_1980,job03_1980,job04_1980,job05_1980,job01_1981,job02_1981,job03_1981,job04_1981,...,job06_2014,job07_2014,job08_2014,job09_2014,job10_2014,job11_2014,job12_2014,job13_2014,job14_2014,job15_2014
0,1,-5,-5,-5,-5,-5,1,-4,-4,-4,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1,2,1,-4,-4,-4,-4,1,-4,-4,-4,...,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4
2,3,1,0,0,-4,-4,-5,-5,-5,-5,...,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4
3,4,-5,-5,-5,-5,-5,-4,-4,-4,-4,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
4,5,1,0,-4,-4,-4,1,0,-4,-4,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5


In [48]:
# which years have cps job identifier?
years = []
for col in cps_id_df.columns[1:]:
    years.append(col[-4:])
    
print set(years)

set(['1986', '1987', '1984', '1985', '1982', '1983', '1980', '1981', '1988', '1989', '2014', '2010', '2012', '1991', '1990', '1993', '1992', '1998', '2002', '2000', '2006', '2004', '2008'])


In [49]:
tenure_df = pd.read_csv('../data/tenure.csv')
tenure_df.head()

,caseid_1979,tenure1_1979,tenure2_1979,tenure3_1979,tenure4_1979,tenure5_1979,tenure1_1980,tenure2_1980,tenure3_1980,tenure4_1980,...,tenure1_2012,tenure2_2012,tenure3_2012,tenure4_2012,tenure5_2012,tenure1_2014,tenure2_2014,tenure3_2014,tenure4_2014,tenure5_2014
0,1,41,21,-4,-4,-4,-5,-5,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1,2,42,-4,-4,-4,-4,95,-4,-4,-4,...,483,-4,-4,-4,-4,590,-4,-4,-4,-4
2,3,21,11,11,-4,-4,7,21,48,-4,...,218,53,-4,-4,-4,329,27,-4,-4,-4
3,4,-4,-4,-4,-4,-4,-5,-5,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
4,5,63,13,-4,-4,-4,113,10,-4,-4,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5


In [50]:
hrrate_df = pd.read_csv('../data/hrrate.csv')
hrrate_df.head()

,caseid_1979,cpshrp_1979,hrp1_1979,hrp2_1979,hrp3_1979,hrp4_1979,hrp5_1979,cpshrp_1980,hrp1_1980,hrp2_1980,...,hrp1_2012,hrp2_2012,hrp3_2012,hrp4_2012,hrp5_2012,hrp1_2014,hrp2_2014,hrp3_2014,hrp4_2014,hrp5_2014
0,1,328,328,-4,-4,-4,-4,-5,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1,2,385,385,-4,-4,-4,-4,457,457,-4,...,1098,-4,-4,-4,-4,1153,-4,-4,-4,-4
2,3,365,365,-4,275,-4,-4,397,397,367,...,1800,-4,-4,-4,-4,1923,721,-4,-4,-4
3,4,-4,-4,-4,-4,-4,-4,-5,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
4,5,310,310,375,-4,-4,-4,333,333,-4,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5


In [51]:
hour_df = pd.read_csv('../data/hours_per_job.csv')
hour_df.head()

,caseid_1979,hours_a_01_1979,hours_a_02_1979,hours_a_03_1979,hours_a_04_1979,hours_a_05_1979,cps_hours_a_1980,hours_a_01_1980,hours_a_02_1980,hours_a_03_1980,...,hours_d_01_2014,hours_d_02_2014,hours_d_03_2014,hours_d_04_2014,hours_d_05_2014,hours_b_01_2014,hours_b_02_2014,hours_b_03_2014,hours_b_04_2014,hours_b_05_2014
0,1,38,15,-4,-4,-4,-5,-5,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1,2,35,-4,-4,-4,-4,35,-4,-4,-4,...,35,-4,-4,-4,-4,0,-4,-4,-4,-4
2,3,35,-3,40,-4,-4,37,-4,40,35,...,40,-4,-4,-4,-4,0,-4,-4,-4,-4
3,4,-4,-4,-4,-4,-4,-5,-5,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
4,5,12,40,-4,-4,-4,12,-4,40,-4,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5


In [52]:
intdate_df = pd.read_csv('../data/int_date.csv')
intdate_df.head()

,caseid_1979,date_m_1979,date_d_1979,date_m_1980,date_d_1980,date_m_1981,date_d_1981,date_m_1982,date_d_1982,date_m_1983,...,date_y_2008,date_d_2010,date_m_2010,date_y_2010,date_d_2012,date_m_2012,date_y_2012,date_d_2014,date_m_2014,date_y_2014
0,1,3,3,-5,-5,6,25,-5,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1,2,2,28,3,3,2,23,2,6,2,...,2008,10,3,2010,3,12,2012,26,12,2014
2,3,2,8,5,20,-5,-5,3,11,3,...,2008,-5,-5,-5,19,3,2013,8,5,2015
3,4,2,8,-5,-5,3,24,3,3,1,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
4,5,4,19,4,3,3,12,3,1,4,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5


### Explore relationship between currently working and CPS job

Most are the same, but not entirely. A CPS job can be a job not currently worked at, and a curently working job can be a non-CPS job. 

In [53]:
tmp = iv_df.merge(current_df, on='caseid_1979')
tmp = tmp.merge(cps_id_df, on='caseid_1979')

In [54]:
job = 2
year = 1990
pd.crosstab(tmp['job%02d_%s'%(job, year)], tmp['current_at_job_%02d_%s'%(job, year)])

current_at_job_02_1990,-5,-4,0,1
job02_1990,,,,
-5,169,0,0,0
-4,0,2604,0,0
0,0,0,716,197
1,0,0,1,1


In [55]:
job = 1
year = 1993
pd.crosstab(tmp['job%02d_%s'%(job, year)], tmp['current_at_job_%02d_%s'%(job, year)])

current_at_job_01_1993,-5,-4,0,1
job01_1993,,,,
-5,105,0,0,0
0,0,848,1,3
1,0,1,365,2365


In [56]:
job = 1
year = 2004
pd.crosstab(tmp['job%02d_%s'%(job, year)], tmp['current_at_job_%02d_%s'%(job, year)])

current_at_job_01_2004,-5,-4,-3,0,1
job01_2004,,,,,
-5,397,0,0,0,0
-4,0,576,1,0,0
0,0,0,0,6,11
1,0,0,12,294,2391


In [57]:
job = 2
year = 2004
pd.crosstab(tmp['job%02d_%s'%(job, year)], tmp['current_at_job_%02d_%s'%(job, year)])

current_at_job_02_2004,-5,-4,-3,0,1
job02_2004,,,,,
-5,397,0,0,0,0
-4,0,2562,1,0,0
0,0,0,1,522,205


### What's the difference between hours_d and hours_all variables?

In [58]:
#cols = [col for col in hour_df.columns if col.endswith('2004')]
hour_df[['hours_all_01_2004', 'hours_d_01_2004', 'hours_all_02_2004', 'hours_d_02_2004', 'hours_all_03_2004', 'hours_d_03_2004']].head(20)

,hours_all_01_2004,hours_d_01_2004,hours_all_02_2004,hours_d_02_2004,hours_all_03_2004,hours_d_03_2004
0,-5,-5,-5,-5,-5,-5
1,16,-4,-4,-4,-4,-4
2,-5,-5,-5,-5,-5,-5
3,-5,-5,-5,-5,-5,-5
4,-5,-5,-5,-5,-5,-5
5,43,43,-4,-4,-4,-4
6,40,-4,-4,-4,-4,-4
7,40,-4,16,-4,-4,-4
8,50,-4,-4,-4,-4,-4
9,-5,-5,-5,-5,-5,-5


In [59]:
hour_df[['hours_a_01_1988', 'hours_c_01_1988', 'hours_d_01_1988', 'hours_a_02_1988', 'hours_c_02_1988', 'hours_d_02_1988', 'hours_a_03_1988', 'hours_c_03_1988', 'hours_d_03_1988']].head(20)

,hours_a_01_1988,hours_c_01_1988,hours_d_01_1988,hours_a_02_1988,hours_c_02_1988,hours_d_02_1988,hours_a_03_1988,hours_c_03_1988,hours_d_03_1988
0,-5,-5,-5,-5,-5,-5,-5,-5,-5
1,-4,-4,-4,15,-4,-4,-4,-4,-4
2,-4,-4,-4,-4,-4,-4,-4,-4,-4
3,-4,-4,-4,5,1,-4,-4,-4,-4
4,-4,-4,-4,60,-4,-4,-4,-4,-4
5,-4,-4,-4,-4,-4,-4,-4,-4,-4
6,-4,-4,-4,-4,-4,-4,-4,-4,-4
7,-4,-4,-4,40,-4,-4,35,-4,-4
8,-4,-4,-4,-4,-4,-4,-4,-4,-4
9,-4,-4,-4,-4,-4,-4,-4,-4,-4


### Get working subsample, hourly rate and tenure variables

Decision rules:

Go to the last survey that is at least a year prior to the birth, provided it is not more than 3 years prior to the birth.

In that wave, look at the jobs the R has held since the previous survey wave (jobs #1-#5). If more than one, tie-break in favor of one that holds currently.

If more than one,  tie-break in favor of one works more hours.

If more than one, tie-break in favor of one with greater tenure.
 
Then just pull the wage and tenure from that job at that wave.

---

There are several variants of job hours variables. 

* If job year is between [1980, 1993], use cps job hours variables. 
* If job year is before (<=) 1987, use hours_a variables. 
* After 1987, if hours_c variable == 1 (i.e. hours_a includes hours worked at home), use hours_a;
otherwise, use hours_d variable. However, 1998 do not have hours_c variables, so modify the above logic:
After 1987, user hours_a variable. if hours_d variable is valid positive hours, update to hours_d. 
* There is no hours_d_01_1993
* After 2004 (inclusive), use hours_all 

In [60]:
def is_cps_job(i, job_y, job_num):
    row = cps_id_df[cps_id_df.caseid_1979==i].iloc[0]
    
    try:
        return row['job%02d_%i'%(job_num, job_y)]==1
            
    except KeyError:
        return False

    return False


def get_hours(i, job_y, job_num):
    
    row = hour_df[hour_df.caseid_1979==i].iloc[0]
    
    res = None
    
    if job_y == 1979: # look for hour_a variables
        res = row['hours_a_%02d_%i'%(job_num, job_y)]
    
    elif job_y <= 1987: # look for cps and hour_a variables
        
        if is_cps_job(i, job_y, job_num):
            res = row['cps_hours_a_%i'%job_y]
        else:
            res = row['hours_a_%02d_%i'%(job_num, job_y)]
        
    elif job_y <= 1993: # look for cps, hour_a, hour_c, and hour_d variables
        
        if is_cps_job(i, job_y, job_num):
            res = row['cps_hours_a_%i'%job_y]
            if row['cps_hours_d_%i'%job_y] >= 0:
                res = row['cps_hours_d_%i'%job_y]

        else:
            res = row['hours_a_%02d_%i'%(job_num, job_y)]
            if row['hours_d_%02d_%i'%(job_num, job_y)] >= 0:
                res = row['hours_d_%02d_%i'%(job_num, job_y)]
            
    elif job_y < 2004:
        res = row['hours_a_%02d_%i'%(job_num, job_y)]
        if row['hours_d_%02d_%i'%(job_num, job_y)] >= 0:
            res = row['hours_d_%02d_%i'%(job_num, job_y)]
            
    else:
        res = row['hours_all_%02d_%i'%(job_num, job_y)]
     
    
    if res == None or res < 0:
        return None
    return res


def get_tenure(row):
    if pd.isnull(row['job_num']):
        return None
    
    res = tenure_df[tenure_df.caseid_1979==row['caseid_1979']].iloc[0]['tenure%s_%s'%(int(row['job_num']), int(row['job_y']))]
    if res < 0:
        return None
    return res


def get_hourly_rate(Id, job_y, job_num):
 
    rate_row = hrrate_df[hrrate_df.caseid_1979==Id].iloc[0]
    
    rate = None
    
    if job_y <= 1994:
        if is_cps_job(rate_row['caseid_1979'], job_y, job_num):
            rate = rate_row['cpshrp_%i'%job_y]
        else:
            rate = rate_row["hrp%i_%i"%(job_num, job_y)]
            
    else:
        rate = rate_row["hrp%i_%i"%(job_num, job_y)]
        
    return rate

In [61]:
def get_job(row):
    """
    Return (job_y, job_num)
    
    If not in sample, return (0, 0)
    If missing, return (None, None)
    """
    
    month = datetime(int(row['childbirth_year'])-1, int(row['childbirth_month']), 1)
    
    int_row = intdate_df[intdate_df.caseid_1979==row['caseid_1979']].iloc[0]
    
    years = range(1979, 1995) + range(1996, 2015, 2)
    
    gap = None
    job_y = None
    for y in reversed(years):
        if int_row['date_m_%s'%y] > 0 and datetime(y, int(int_row['date_m_%s'%y]), 1) <= month:
            gap = diff_month(month, datetime(y, int(int_row['date_m_%s'%y]), 1))
            job_y = y
            break
    
    if gap > 23: # more than 3 years ago
        return (0, 0)
        
        
    wages = [get_hourly_rate(row['caseid_1979'], job_y, j) for j in range(1, 6)]
    has_missing = [(val < 0 and val != -4) for val in wages]
    has_valid_wage = [(val >= 0) for val in wages]
    
    if wages.count(-4) == 5: # not in subsample
        return (0, 0)
    
    elif sum(has_missing) > 0: # there's missing - NONE
        print row['caseid_1979'], wages
        return (None, None)
        
    elif sum(has_valid_wage) == 1: # only one job
        return (job_y, has_valid_wage.index(1)+1)

    else:
        jobs = [j+1 for j in range(5) if has_valid_wage[j]==1]
        
        # use 'current at job' to break tie
        current_row = current_df[current_df.caseid_1979==row['caseid_1979']].iloc[0]
        current = [current_row['current_at_job_%02d_%i'%(j, job_y)] for j in jobs]
        
        if current.count(1) == 1:
            return (job_y, jobs[current.index(1)])
        
        
        # next, use hours to break tie
        if 1 in current:
            jobs = [jobs[j] for j in range(len(jobs)) if current[j]==1]
        
        hours = [get_hours(row['caseid_1979'], job_y, j) for j in jobs]
        
        if None in hours:
            print row['caseid_1979'], job_y, hours
        
        if hours.count(max(hours)) == 1:
            return (job_y, jobs[hours.index(max(hours))])


        # next, use tenure to break tie
        jobs = [jobs[j] for j in range(len(jobs)) if hours[j]==max(hours)]
        
        tenure_row = tenure_df[tenure_df.caseid_1979==row['caseid_1979']].iloc[0]
        tenures = [tenure_row['tenure%s_%s'%(j, job_y)] for j in jobs]
        
        if tenures.count(max(tenures)) == 1:
            return (job_y, jobs[tenures.index(max(tenures))])
        
        
        print 'uncaptured:', row['caseid_1979']
    


    
res = sub_iv_df.apply(get_job, axis=1)   
sub_iv_df['job_y'] = map(itemgetter(0), res)
sub_iv_df['job_num'] = map(itemgetter(1), res)
sub_iv_df.head()

452.0 1992 [20, None]
9219.0 1998 [None, 32]


,caseid_1979,childbirth_year,childbirth_month,y_dob,m_dob,race,age,educ,marstat,mom_worked,...,work35,womensroles,ideal,religion,religion_freq,hours_2y_to_1y,hours_1y_to_birth,married_sample,job_y,job_num
0,2,1993,3,1959,1,3,34,2,1.0,0.0,...,1.0,3.4,3.0,2.0,3.0,1820.0,1820.0,1,1991,1
1,3,1981,6,1961,8,3,19,1,1.0,1.0,...,0.0,3.0,3.0,2.0,1.0,1440.0,NaN,1,1980,1
2,4,1980,8,1962,8,3,17,1,0.0,1.0,...,0.0,2.2,3.0,2.0,1.0,0.0,NaN,0,0,0
3,16,1990,2,1958,10,3,31,3,1.0,0.0,...,1.0,2.2,2.0,2.0,1.0,2641.0,2005.0,1,1988,1
4,19,1987,11,1957,12,3,29,2,1.0,0.0,...,1.0,2.2,4.0,2.0,1.0,138.0,1490.0,0,1986,1


In [62]:
sub_iv_df['work_sample'] = sub_iv_df.job_num.map(lambda x: int(x!=0))
#work_sample = sub_iv_df[sub_iv_df.job_num!=0]
work_sample = sub_iv_df[sub_iv_df.work_sample==1]
print work_sample.shape

(2180, 36)


In [63]:
def get_tenure(row):
    if pd.isnull(row['job_num']):
        return None
    
    res = tenure_df[tenure_df.caseid_1979==row['caseid_1979']].iloc[0]['tenure%s_%s'%(int(row['job_num']), int(row['job_y']))]
    if res < 0:
        return None
    return res


def get_hourly_rate2(row):
    
    rate = get_hourly_rate(row['caseid_1979'], int(row['job_y']), int(row['job_num']))
 
    if rate < 0:
        return None
        
    return rate/100.


work_sample['tenure'] = work_sample.apply(get_tenure, axis=1)
work_sample['rate'] = work_sample.apply(get_hourly_rate2, axis=1)
work_sample['rate_adj'] = work_sample.apply(lambda row: convert_to_current_dollar(row['rate'], row['job_y']), axis=1)
work_sample.head()

/Users/xzhuo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/xzhuo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/xzhuo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable

,caseid_1979,childbirth_year,childbirth_month,y_dob,m_dob,race,age,educ,marstat,mom_worked,...,religion_freq,hours_2y_to_1y,hours_1y_to_birth,married_sample,job_y,job_num,work_sample,tenure,rate,rate_adj
0,2,1993,3,1959,1,3,34,2,1.0,0.0,...,3.0,1820.0,1820.0,1,1991,1,1,689.0,10.98,19.3248
1,3,1981,6,1961,8,3,19,1,1.0,1.0,...,1.0,1440.0,NaN,1,1980,1,1,7.0,3.97,12.1085
3,16,1990,2,1958,10,3,31,3,1.0,0.0,...,1.0,2641.0,2005.0,1,1988,1,1,106.0,15.38,31.5290
4,19,1987,11,1957,12,3,29,2,1.0,0.0,...,1.0,138.0,1490.0,0,1986,1,1,16.0,5.65,12.2040
5,20,1990,8,1959,11,3,30,4,1.0,0.0,...,2.0,2080.0,2080.0,1,1989,1,1,275.0,15.86,31.0856


In [65]:
work_sample.to_csv('../data/iv_df_seq14_work.csv', index=False)

## Put all datasets together

In [2]:
df = pd.read_csv('../data/iv_df_seq14.csv')
df_post1979 = pd.read_csv('../data/iv_df_seq14_post1979.csv')
df_married = pd.read_csv('../data/iv_df_seq14_married.csv')
df_work = pd.read_csv('../data/iv_df_seq14_work.csv')

In [3]:
new_cols = [col for col in df_post1979.columns if not col in df.columns]
new_cols.append('caseid_1979')
df = df.merge(df_post1979[new_cols], on='caseid_1979', how='left')
df.head()

,caseid_1979,childbirth_year,childbirth_month,y_dob,m_dob,race,age,educ,marstat,mom_worked,...,num_miss,num_nonemp_spells,post1979_sample,work35,womensroles,ideal,religion,religion_freq,hours_2y_to_1y,hours_1y_to_birth
0,2,1993,3,1959,1,3,34,2,1.0,0.0,...,0,1,1.0,1.0,3.4,3.0,2.0,3.0,1820.0,1820.0
1,3,1981,6,1961,8,3,19,1,1.0,1.0,...,0,7,1.0,0.0,3.0,3.0,2.0,1.0,1440.0,NaN
2,4,1980,8,1962,8,3,17,1,0.0,1.0,...,0,5,1.0,0.0,2.2,3.0,2.0,1.0,0.0,NaN
3,8,1976,3,1958,7,3,17,1,1.0,1.0,...,21,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16,1990,2,1958,10,3,31,3,1.0,0.0,...,0,0,1.0,1.0,2.2,2.0,2.0,1.0,2641.0,2005.0


In [4]:
new_cols = [col for col in df_married.columns if not col in df.columns]
new_cols.append('caseid_1979')
df = df.merge(df_married[new_cols], on='caseid_1979', how='left')
df.head()

,caseid_1979,childbirth_year,childbirth_month,y_dob,m_dob,race,age,educ,marstat,mom_worked,...,ideal,religion,religion_freq,hours_2y_to_1y,hours_1y_to_birth,married_sample,spouse_earning,spouse_earning_adj,spouse_hours,spouse_hours_gp
0,2,1993,3,1959,1,3,34,2,1.0,0.0,...,3.0,2.0,3.0,1820.0,1820.0,1.0,40000.0,68800.0,35.0,1.0
1,3,1981,6,1961,8,3,19,1,1.0,1.0,...,3.0,2.0,1.0,1440.0,NaN,1.0,NaN,NaN,NaN,NaN
2,4,1980,8,1962,8,3,17,1,0.0,1.0,...,3.0,2.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,8,1976,3,1958,7,3,17,1,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16,1990,2,1958,10,3,31,3,1.0,0.0,...,2.0,2.0,1.0,2641.0,2005.0,1.0,33000.0,64680.0,60.0,2.0


In [5]:
new_cols = [col for col in df_work.columns if not col in df.columns]
new_cols.append('caseid_1979')
df = df.merge(df_work[new_cols], on='caseid_1979', how='left')
df.head()

,caseid_1979,childbirth_year,childbirth_month,y_dob,m_dob,race,age,educ,marstat,mom_worked,...,spouse_earning,spouse_earning_adj,spouse_hours,spouse_hours_gp,job_y,job_num,work_sample,tenure,rate,rate_adj
0,2,1993,3,1959,1,3,34,2,1.0,0.0,...,40000.0,68800.0,35.0,1.0,1991.0,1.0,1.0,689.0,10.98,19.3248
1,3,1981,6,1961,8,3,19,1,1.0,1.0,...,NaN,NaN,NaN,NaN,1980.0,1.0,1.0,7.0,3.97,12.1085
2,4,1980,8,1962,8,3,17,1,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8,1976,3,1958,7,3,17,1,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16,1990,2,1958,10,3,31,3,1.0,0.0,...,33000.0,64680.0,60.0,2.0,1988.0,1.0,1.0,106.0,15.38,31.5290


In [6]:
# replace None in subsample dummies to 0

def recode_dummy(x):
    if pd.isnull(x):
        return 0
    elif x == 1:
        return 1
    else:
        raise Exception, 'unknown value %s'%x
        
df.post1979_sample = df.post1979_sample.map(recode_dummy)
df.married_sample = df.married_sample.map(recode_dummy)
df.work_sample = df.work_sample.map(recode_dummy)

In [7]:
df.post1979_sample.value_counts()

1    2706
0     982
Name: post1979_sample, dtype: int64

In [8]:
df.married_sample.value_counts()

0    1921
1    1767
Name: married_sample, dtype: int64

In [9]:
df.work_sample.value_counts()

1    2180
0    1508
Name: work_sample, dtype: int64

In [10]:
df.to_csv('../data/iv_df_seq14_all.csv', index=False)